# Predicting Match Outcomes

In [1]:
#SQL Imports
import mysql.connector
#Pandas imports
import pandas as pd
import datetime

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [3]:
query = pd.read_sql_query('''
    SELECT * 
    FROM TeamMatchStats
''', mydb)

In [4]:
match_stat_df = pd.DataFrame(query)

In [5]:
match_stat_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,prop_1,prop_2,hooker,sr_1,sr_2,lock_1,int_1,int_2,int_3,int_4
0,1,1,14,1,1,34,5,4,3,0.8000,...,404,396,391,407,406,483,1626,402,397,219
1,2,1,1,14,0,12,2,1,1,0.5000,...,11,500,499,2,10,269,3,400,9,1624
2,3,2,8,6,1,19,3,3,0,1.0000,...,192,503,201,196,188,197,190,429,194,505
3,4,2,6,8,0,18,3,2,1,0.6667,...,510,509,153,511,155,512,516,309,513,327
4,5,3,10,4,1,20,4,1,1,0.2500,...,268,272,265,264,375,266,260,276,267,261


# Prediction Algorithm

Things to try:
1. Want to create scoring function (sklearn.metrics import f1_score, make_scorer, classification_report)

2. Random Forest
3. Neural Network (10-3-2)

Features to try:
- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- Tackle Percentages
- Factor in the strength of opposition


In [6]:
correlation_df = match_stat_df
col = list(match_stat_df.columns)[35:52]
correlation_df = match_stat_df.drop(col, axis=1)
correlation_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,conversions,penalty_goals,conversion_percentage,...,ineffective_tackles,tackle_percentage,kicks,kicking_metres,metres_per_kick,errors,handling_errors,penalties,sin_bins,send_offs
0,1,1,14,1,1,34,5,4,3,0.8000,...,14,0.8881,17,478,28.1176,10,9,7,0,0
1,2,1,1,14,0,12,2,1,1,0.5000,...,15,0.8715,14,382,27.2857,13,11,9,0,0
2,3,2,8,6,1,19,3,3,0,1.0000,...,27,0.8465,21,538,25.6190,6,6,9,0,0
3,4,2,6,8,0,18,3,2,1,0.6667,...,5,0.8883,13,284,21.8462,16,10,11,0,0
4,5,3,10,4,1,20,4,1,1,0.2500,...,19,0.8774,13,252,19.3846,15,10,6,0,0


In [7]:
correlation_df = correlation_df.drop(columns=['line_engaged_runs'])

In [8]:
import seaborn as sns
corr = correlation_df.corr()
corr['is_winner']
stats = corr[(corr['is_winner'] > .2) | (corr['is_winner'] < -.2)]
stats['is_winner']
#corr.style.background_gradient(cmap='coolwarm')

is_winner              1.000000
points                 0.616395
tries                  0.563085
conversions            0.495853
penalty_goals          0.331492
field_goals            0.213446
runs                   0.291058
run_metres             0.387814
kick_return_metres     0.257286
post_contact_metres    0.253308
line_breaks            0.372330
tackle_breaks          0.291125
hit_ups                0.295609
tackles               -0.330251
missed_tackles        -0.290167
kicks                  0.236040
kicking_metres         0.247338
errors                -0.236594
handling_errors       -0.203090
Name: is_winner, dtype: float64

Columns to take into consideration
From highly correlated stats:

- points (allowed)
- tries (allowed)
- penalty_goals
- run_metres (allowed)
- runs
- runs_per_metre (allowed)
- line_breaks (allowed)
- tackle_breaks
- tackles (allowed)
- missed_tackles
- kicking_metres
- errors

In [9]:
stat_columns = ['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points', 'tries', 'penalty_goals',
                     'run_metres', 'runs', 'line_breaks', 'tackle_breaks', 'tackles', 'missed_tackles',
                     'kicking_metres', 'errors']
results_df = match_stat_df[stat_columns]
results_df['metres_per_run'] = results_df['run_metres'] / results_df['runs']
results_df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,line_breaks,tackle_breaks,tackles,missed_tackles,kicking_metres,errors,metres_per_run
0,1,1,14,1,1,34,5,3,1618,168,5,35,238,16,478,10,9.630952
1,2,1,1,14,0,12,2,1,1076,130,2,16,339,35,382,13,8.276923
2,3,2,8,6,1,19,3,0,1425,192,3,41,342,35,538,6,7.421875
3,4,2,6,8,0,18,3,1,1648,178,4,35,366,41,284,16,9.258427
4,5,3,10,4,1,20,4,1,1348,165,3,37,272,19,252,15,8.169697


columns to create

- points (allowed)
- tries (allowed)
- penalty_goals
- run_metres (allowed)
- runs
- runs_per_metre (allowed)
- line_breaks (allowed)
- tackle_breaks
- tackles (allowed)
- missed_tackles
- kicking_metres
- errors

- tackle_percentage
- metres_per_run

- point_differential
- run_metre differential
- 

- home field advantage
- form in last X matches
- Points differential last X matches
- Points scored last X matches
- Points allowed last X matches
- Running metre differential last X matches
- line_breaks last X matches
- errors last X matches
- Tackle Percentages
- Strength of opposition metric

In [10]:
columns = [*results_df.columns.tolist(),'opp_points', 'opp_tries', 'opp_run_metres',
           'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff', 'run_metre_diff']
results_df = results_df.reindex(columns=columns, fill_value=0)
#print(predictions_df)

for match in list(results_df['match_id'].unique()):
    match_df = results_df[results_df['match_id'] == match]
    #print(match_df)
    for team in list(match_df['team_id']):
        team_data = match_df[match_df['team_id'] == team]
        opp_data = match_df[match_df['team_id'] != team]
        
        opp_points = opp_data['points'].iloc[0]
        opp_tries = opp_data['tries'].iloc[0]
        opp_run_metres = opp_data['run_metres'].iloc[0]
        opp_metres_per_run = opp_data['metres_per_run'].iloc[0]
        opp_line_breaks = opp_data['line_breaks'].iloc[0]
        opp_tackles = opp_data['tackles'].iloc[0]
        
        point_diff = team_data['points'].iloc[0] - opp_points
        run_metre_diff = team_data['run_metres'].iloc[0] - opp_run_metres
        
        results_df.loc[team_data.index, 'opp_points'] = opp_points
        results_df.loc[team_data.index, 'opp_tries'] = opp_tries
        results_df.loc[team_data.index, 'opp_run_metres'] = opp_run_metres
        results_df.loc[team_data.index, 'opp_metres_per_run'] = opp_metres_per_run
        results_df.loc[team_data.index, 'opp_line_breaks'] = opp_line_breaks
        results_df.loc[team_data.index, 'opp_tackles'] = opp_tackles
        results_df.loc[team_data.index, 'point_diff'] = point_diff
        results_df.loc[team_data.index, 'run_metre_diff'] = run_metre_diff
          
results_df.head()

,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,...,errors,metres_per_run,opp_points,opp_tries,opp_run_metres,opp_metres_per_run,opp_line_breaks,opp_tackles,point_diff,run_metre_diff
0,1,1,14,1,1,34,5,3,1618,168,...,10,9.630952,12,2,1076,8.276923,2,339,22,542
1,2,1,1,14,0,12,2,1,1076,130,...,13,8.276923,34,5,1618,9.630952,5,238,-22,-542
2,3,2,8,6,1,19,3,0,1425,192,...,6,7.421875,18,3,1648,9.258427,4,366,1,-223
3,4,2,6,8,0,18,3,1,1648,178,...,16,9.258427,19,3,1425,7.421875,3,342,-1,223
4,5,3,10,4,1,20,4,1,1348,165,...,15,8.169697,14,2,1144,8.537313,1,323,6,204


In [11]:
def compute_past_x_match_avg(x, stat, data):
    x_match_avg = []
    for index in range(len(data)):
        if index <= (x - 1):
            start = 0
        else:
            start = (index - x)
        past_x_matches = data.iloc[start : index]
        avg = past_x_matches[stat].mean()
        x_match_avg.append(avg)
    return pd.Series(x_match_avg).values

In [12]:
columns = results_df.columns.tolist()
stats = ['is_winner', 'points', 'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
         'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres', 'errors']

for stat in stats:
    for x in [4, 8]:
        columns.append('avg_' + stat + '_past_' + str(x) + '_matches')
results_df = results_df.reindex(columns=columns, fill_value=0)

for team in list(results_df['team_id'].unique()):
    team_df = results_df[results_df['team_id'] == team]
    team_df = team_df.reset_index()
    for stat in stats:
        for x in [4, 8]:
            new_column = 'avg_' + stat + '_past_' + str(x) + '_matches'
            team_df[new_column] = compute_past_x_match_avg(x, stat, team_df)
    team_df = team_df.set_index('index')
    results_df.loc[results_df['team_id'] == team] = team_df
    #print(predictions_df[predictions_df['team_id'] == team])
print(results_df[results_df['team_id'] == 12])

      id  match_id  team_id  opponent_id  is_winner  points  tries  \
12    13         7       12           11          1      24      3   
24    25        13       12           13          1      18      3   
35    36        18       12            3          0      18      3   
49    50        25       12           10          1      33      4   
79    80        40       12           11          1      12      1   
92    93        47       12            5          1      35      6   
111  112        56       12            4          0      22      4   
120  121        61       12            3          1      22      3   
132  133        67       12           10          0      20      4   
147  148        74       12            8          1      29      4   
160  161        81       12           16          1      16      2   
186  187        94       12           14          1      28      4   
201  202       101       12            2          1      23      3   
221  222       111  

In [13]:
results_df.tail()

,id,match_id,team_id,opponent_id,is_winner,points,tries,penalty_goals,run_metres,runs,...,avg_tackle_breaks_past_4_matches,avg_tackle_breaks_past_8_matches,avg_tackles_past_4_matches,avg_tackles_past_8_matches,avg_missed_tackles_past_4_matches,avg_missed_tackles_past_8_matches,avg_kicking_metres_past_4_matches,avg_kicking_metres_past_8_matches,avg_errors_past_4_matches,avg_errors_past_8_matches
507,508,254,16,15,0,12,2,0,1479,161,...,32.25,29.875,367.75,361.500,25.00,26.625,537.50,542.375,10.00,9.750
508,509,255,9,8,0,18,3,0,1375,162,...,36.75,32.750,368.50,351.250,32.75,29.000,467.00,481.375,9.00,9.500
509,510,255,8,9,1,36,5,4,1679,167,...,34.75,29.250,356.00,339.250,33.50,36.125,474.25,481.000,10.50,11.000
510,511,256,11,14,1,32,5,2,1670,178,...,29.00,27.625,341.00,334.750,38.25,34.250,447.00,473.750,12.25,12.125
511,512,256,14,11,0,18,3,1,1419,159,...,30.25,33.375,356.50,354.375,22.50,24.250,683.25,625.750,10.25,10.500


In [14]:
query = pd.read_sql_query('''
    SELECT * 
    FROM Matches
    WHERE id < 257;
''', mydb)
predictions_df = pd.DataFrame(query)
predictions_df.tail()

,id,date,round,home_team_id,home_score,away_team_id,away_score,winner,is_draw,stadium_id,weather,url,home_odds,draw_odds,away_odds,is_playoff
251,252,2019-05-04,8,2,30,12,12,2,0,36,None,http://www.nrl.com/draw/nrl-premiership/2019/r...,0.5714,0.0504,0.463,0
252,253,2019-05-04,8,6,18,3,10,6,0,13,None,http://www.nrl.com/draw/nrl-premiership/2019/r...,0.5618,0.0534,0.4739,0
253,254,2019-05-04,8,15,42,16,12,15,0,29,None,http://www.nrl.com/draw/nrl-premiership/2019/r...,0.8264,0.0418,0.2174,0
254,255,2019-05-05,8,9,18,8,36,8,0,30,None,http://www.nrl.com/draw/nrl-premiership/2019/r...,0.6135,0.05,0.4237,0
255,256,2019-05-05,8,11,32,14,18,11,0,35,None,http://www.nrl.com/draw/nrl-premiership/2019/r...,0.5525,0.052,0.4808,0


In [15]:
predictions_df = predictions_df[['id', 'round', 'home_team_id', 'home_score', 'away_team_id', 'away_score', 'winner', 'home_odds', 'away_odds']]

In [16]:
columns = list(results_df.drop(columns=['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points',
                                    'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
                                    'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres',
                                    'errors', 'metres_per_run', 'opp_points', 'opp_tries', 'opp_run_metres',
                                    'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff',
                                    'run_metre_diff']))

prediction_columns = predictions_df.columns.tolist()
for team in ['h', 'a']:
    for column in columns:
        prediction_columns.append(team + '_' + column)
print(prediction_columns)
predictions_df = predictions_df.reindex(columns=prediction_columns, fill_value=0)
predictions_df.head()

['id', 'round', 'home_team_id', 'home_score', 'away_team_id', 'away_score', 'winner', 'home_odds', 'away_odds', 'h_avg_is_winner_past_4_matches', 'h_avg_is_winner_past_8_matches', 'h_avg_points_past_4_matches', 'h_avg_points_past_8_matches', 'h_avg_tries_past_4_matches', 'h_avg_tries_past_8_matches', 'h_avg_penalty_goals_past_4_matches', 'h_avg_penalty_goals_past_8_matches', 'h_avg_run_metres_past_4_matches', 'h_avg_run_metres_past_8_matches', 'h_avg_runs_past_4_matches', 'h_avg_runs_past_8_matches', 'h_avg_line_breaks_past_4_matches', 'h_avg_line_breaks_past_8_matches', 'h_avg_tackle_breaks_past_4_matches', 'h_avg_tackle_breaks_past_8_matches', 'h_avg_tackles_past_4_matches', 'h_avg_tackles_past_8_matches', 'h_avg_missed_tackles_past_4_matches', 'h_avg_missed_tackles_past_8_matches', 'h_avg_kicking_metres_past_4_matches', 'h_avg_kicking_metres_past_8_matches', 'h_avg_errors_past_4_matches', 'h_avg_errors_past_8_matches', 'a_avg_is_winner_past_4_matches', 'a_avg_is_winner_past_8_matche

,id,round,home_team_id,home_score,away_team_id,away_score,winner,home_odds,away_odds,h_avg_is_winner_past_4_matches,...,a_avg_tackle_breaks_past_4_matches,a_avg_tackle_breaks_past_8_matches,a_avg_tackles_past_4_matches,a_avg_tackles_past_8_matches,a_avg_missed_tackles_past_4_matches,a_avg_missed_tackles_past_8_matches,a_avg_kicking_metres_past_4_matches,a_avg_kicking_metres_past_8_matches,a_avg_errors_past_4_matches,a_avg_errors_past_8_matches
0,1,1,14,34,1,12,14,0.5882,0.4386,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,8,19,6,18,8,0.5181,0.5128,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,10,20,4,14,10,0.5988,0.4329,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,16,10,15,8,16,0.25,0.7874,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,13,20,9,32,9,0.6803,0.3509,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
for match in predictions_df.iterrows():
    match = match[1]
    home_team = match['home_team_id']
    away_team = match['away_team_id']
    match_result_data = results_df.loc[results_df['match_id'] == match['id']]
    home_data = match_result_data[match_result_data['team_id'] == home_team]
    away_data = match_result_data[match_result_data['team_id'] == away_team]
    for team in ['h', 'a']:
        if team == 'h':
            df = home_data
        else:
            df = away_data
        #print(df)
        df = df.drop(columns=['id', 'match_id', 'team_id', 'opponent_id', 'is_winner', 'points',
                              'tries', 'penalty_goals', 'run_metres', 'runs', 'line_breaks',
                              'tackle_breaks', 'tackles', 'missed_tackles', 'kicking_metres',
                              'errors', 'metres_per_run', 'opp_points', 'opp_tries', 'opp_run_metres',
                              'opp_metres_per_run', 'opp_line_breaks', 'opp_tackles', 'point_diff',
                              'run_metre_diff'])
        for column in list(df.columns):
            col = team + '_' + column
            #print(df.iloc[0][column])
            match[col] = df.iloc[0][column]
            #print('\n')
            #print(df.iloc[0][column])
    #print(match.values)
    #print('\n')
    predictions_df.loc[predictions_df['id'] == match['id']] = match.values
print(predictions_df.head(10))

   id  round  home_team_id  home_score  away_team_id  away_score  winner  \
0   1      1            14          34             1          12      14   
1   2      1             8          19             6          18       8   
2   3      1            10          20             4          14      10   
3   4      1            16          10            15           8      16   
4   5      1            13          20             9          32       9   
5   6      1             3          18             7          36       7   
6   7      1            12          24            11          14      12   
7   8      1             5          30             2          28       5   
8   9      2             4          16            14          20      14   
9  10      2            15          30             3          12      15   

  home_odds away_odds  h_avg_is_winner_past_4_matches  ...  \
0    0.5882    0.4386                             NaN  ...   
1    0.5181    0.5128                  

## want to predict if home team wins
1 = home_team wins, 0 = away_team wins

In [18]:
predictions_df.head()

,id,round,home_team_id,home_score,away_team_id,away_score,winner,home_odds,away_odds,h_avg_is_winner_past_4_matches,...,a_avg_tackle_breaks_past_4_matches,a_avg_tackle_breaks_past_8_matches,a_avg_tackles_past_4_matches,a_avg_tackles_past_8_matches,a_avg_missed_tackles_past_4_matches,a_avg_missed_tackles_past_8_matches,a_avg_kicking_metres_past_4_matches,a_avg_kicking_metres_past_8_matches,a_avg_errors_past_4_matches,a_avg_errors_past_8_matches
0,1,1,14,34,1,12,14,0.5882,0.4386,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,8,19,6,18,8,0.5181,0.5128,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,10,20,4,14,10,0.5988,0.4329,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,16,10,15,8,16,0.25,0.7874,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,13,20,9,32,9,0.6803,0.3509,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
predictions_df['home_winner'] = predictions_df.apply(lambda x: x['home_team_id'] == x['winner'], axis=1)

In [20]:
predictions_df = predictions_df.rename(columns={'home_odds': 'h_odds', 'away_odds': 'a_odds'})

In [21]:
comparison_cols = list(predictions_df.drop(columns=['id', 'round', 'home_team_id', 'home_score', 'away_team_id',
                                                                  'away_score', 'winner', 'home_winner']).columns)
stat_list = []
for column in comparison_cols:
    stat = column.split('_', 1)[1]
    if stat not in stat_list:
        stat_list.append(stat)
#stat_list

In [22]:
def compute_home_differential(row, column):
    h_col = 'h_' + column
    a_col = 'a_' + column
    return float(row[h_col]) - float(row[a_col])

diff_cols = []
for stat in stat_list:
    diff_col = 'diff_' + stat
    diff_cols.append(diff_col)
    predictions_df[diff_col] = predictions_df.apply(lambda x: compute_home_differential(x, stat), axis=1)
#predictions_df.head(10)
print(diff_cols)

['diff_odds', 'diff_avg_is_winner_past_4_matches', 'diff_avg_is_winner_past_8_matches', 'diff_avg_points_past_4_matches', 'diff_avg_points_past_8_matches', 'diff_avg_tries_past_4_matches', 'diff_avg_tries_past_8_matches', 'diff_avg_penalty_goals_past_4_matches', 'diff_avg_penalty_goals_past_8_matches', 'diff_avg_run_metres_past_4_matches', 'diff_avg_run_metres_past_8_matches', 'diff_avg_runs_past_4_matches', 'diff_avg_runs_past_8_matches', 'diff_avg_line_breaks_past_4_matches', 'diff_avg_line_breaks_past_8_matches', 'diff_avg_tackle_breaks_past_4_matches', 'diff_avg_tackle_breaks_past_8_matches', 'diff_avg_tackles_past_4_matches', 'diff_avg_tackles_past_8_matches', 'diff_avg_missed_tackles_past_4_matches', 'diff_avg_missed_tackles_past_8_matches', 'diff_avg_kicking_metres_past_4_matches', 'diff_avg_kicking_metres_past_8_matches', 'diff_avg_errors_past_4_matches', 'diff_avg_errors_past_8_matches']


In [23]:
cols = ['id', 'round', 'home_team_id', 'away_team_id', 'winner', 'home_winner']
cols.extend(diff_cols)
print(cols)
final_predictions = predictions_df[cols] 
final_predictions.head()

['id', 'round', 'home_team_id', 'away_team_id', 'winner', 'home_winner', 'diff_odds', 'diff_avg_is_winner_past_4_matches', 'diff_avg_is_winner_past_8_matches', 'diff_avg_points_past_4_matches', 'diff_avg_points_past_8_matches', 'diff_avg_tries_past_4_matches', 'diff_avg_tries_past_8_matches', 'diff_avg_penalty_goals_past_4_matches', 'diff_avg_penalty_goals_past_8_matches', 'diff_avg_run_metres_past_4_matches', 'diff_avg_run_metres_past_8_matches', 'diff_avg_runs_past_4_matches', 'diff_avg_runs_past_8_matches', 'diff_avg_line_breaks_past_4_matches', 'diff_avg_line_breaks_past_8_matches', 'diff_avg_tackle_breaks_past_4_matches', 'diff_avg_tackle_breaks_past_8_matches', 'diff_avg_tackles_past_4_matches', 'diff_avg_tackles_past_8_matches', 'diff_avg_missed_tackles_past_4_matches', 'diff_avg_missed_tackles_past_8_matches', 'diff_avg_kicking_metres_past_4_matches', 'diff_avg_kicking_metres_past_8_matches', 'diff_avg_errors_past_4_matches', 'diff_avg_errors_past_8_matches']


,id,round,home_team_id,away_team_id,winner,home_winner,diff_odds,diff_avg_is_winner_past_4_matches,diff_avg_is_winner_past_8_matches,diff_avg_points_past_4_matches,...,diff_avg_tackle_breaks_past_4_matches,diff_avg_tackle_breaks_past_8_matches,diff_avg_tackles_past_4_matches,diff_avg_tackles_past_8_matches,diff_avg_missed_tackles_past_4_matches,diff_avg_missed_tackles_past_8_matches,diff_avg_kicking_metres_past_4_matches,diff_avg_kicking_metres_past_8_matches,diff_avg_errors_past_4_matches,diff_avg_errors_past_8_matches
0,1,1,14,1,14,True,0.1496,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,8,6,8,True,0.0053,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,10,4,10,True,0.1659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,16,15,16,True,-0.5374,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,13,9,9,False,0.3294,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
#final_predictions.corr()
final_predictions = final_predictions.dropna().reset_index()
print(final_predictions.shape)
final_predictions.head()

(248, 33)


,level_0,index,id,round,home_team_id,away_team_id,winner,home_winner,diff_odds,diff_avg_is_winner_past_4_matches,...,diff_avg_tackle_breaks_past_4_matches,diff_avg_tackle_breaks_past_8_matches,diff_avg_tackles_past_4_matches,diff_avg_tackles_past_8_matches,diff_avg_missed_tackles_past_4_matches,diff_avg_missed_tackles_past_8_matches,diff_avg_kicking_metres_past_4_matches,diff_avg_kicking_metres_past_8_matches,diff_avg_errors_past_4_matches,diff_avg_errors_past_8_matches
0,0,8,9,2,4,14,14,False,-0.0079,-1.0,...,-16.0,-16.0,85.0,85.0,21.0,21.0,-185.0,-185.0,4.0,4.0
1,1,9,10,2,15,3,15,True,0.4620,0.0,...,-6.0,-6.0,48.0,48.0,-4.0,-4.0,234.0,234.0,0.0,0.0
2,2,10,11,2,1,10,1,True,-0.1330,-1.0,...,-21.0,-21.0,67.0,67.0,16.0,16.0,130.0,130.0,-2.0,-2.0
3,3,11,12,2,9,5,9,True,0.4848,0.0,...,-2.0,-2.0,35.0,35.0,-2.0,-2.0,27.0,27.0,-4.0,-4.0
4,4,12,13,2,12,13,12,True,0.3989,1.0,...,-2.0,-2.0,15.0,15.0,1.0,1.0,184.0,184.0,-4.0,-4.0


In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

features_df = final_predictions.drop(columns=['index', 'id', 'round', 'home_team_id', 'away_team_id', 'winner', 'home_winner'])
target = final_predictions['home_winner']
X_train = features_df[:208]
y_train = target[:208]
X_test = features_df[208:]
y_test = target[208:]

reg = LinearRegression()
reg.fit(X_train, y_train)
predictions = reg.predict(X_test)


In [39]:
df = pd.DataFrame({'winner': y_test, 'prediction': predictions})
df

,winner,prediction
208,True,0.601431
209,True,0.590295
210,True,0.417159
211,True,0.503935
212,False,0.159388
213,True,0.488270
214,True,0.357973
215,False,0.516228
216,False,0.720260
217,True,0.182868


In [ ]:
mycursor.close()